In [13]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np

In [14]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [15]:
stop_times_full = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [16]:
stop_times_full

,trip_id,stop_sequence_new,stop_sequence,hr,min,INT_ID
0,32828689,1,1,5,15,13466640.0
1,32828689,2,2,5,16,13466531.0
2,32828689,3,3,5,16,13466677.0
3,32828689,4,4,5,17,13466754.0
4,32828689,5,5,5,18,13466699.0
...,...,...,...,...,...,...
3959705,32970576,7,8,25,56,13461602.0
3959706,32970576,8,9,25,57,13461912.0
3959707,32970576,9,10,25,57,13462284.0
3959708,32970576,10,11,25,58,13462427.0


In [17]:
stop_times_full['hr'] = np.where(stop_times_full['hr'] > 23, stop_times_full['hr'] - 24, stop_times_full['hr'])
stop_times_full_o = stop_times_full.copy()
stop_times_full_d = stop_times_full.copy()
stop_times_full_d['stop_sequence_new'] = stop_times_full_d['stop_sequence_new'] - 1
stop_times_od = stop_times_full_o.merge(stop_times_full_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']])

In [18]:
stop_times_od.groupby(['hr_o', 'INT_ID_o', 'INT_ID_d']).count().reset_index().sort_values(
    by = 'trip_id', ascending = False)[['INT_ID_o', 'INT_ID_d', 'trip_id']].drop_duplicates(
    subset = ['INT_ID_o', 'INT_ID_d'], keep = 'first').head(10)

,INT_ID_o,INT_ID_d,trip_id
107013,304.0,305.0,84
46283,305.0,304.0,65
49123,13452476.0,228.0,61
38448,228.0,13452476.0,60
53043,13468120.0,13468053.0,51
53063,13468224.0,13468120.0,51
52974,13467722.0,13467654.0,50
52953,13467654.0,13467505.0,50
53034,13468053.0,13467971.0,50
38246,116.0,13445250.0,49


In [19]:
max_freq = stop_times_od.groupby(['hr_o', 'INT_ID_o', 'INT_ID_d']).count().reset_index().sort_values(
    by = 'trip_id', ascending = False)[['INT_ID_o', 'INT_ID_d', 'trip_id']].drop_duplicates(
    subset = ['INT_ID_o', 'INT_ID_d'], keep = 'first')['trip_id'].max()

max_freq

84

In [20]:
stop_times_od[(stop_times_od['INT_ID_o'] == 304) & (stop_times_od['INT_ID_d'] == 305)][['route_short_name']].drop_duplicates()

,route_short_name
50561,134
199941,129
209275,131
209949,9131
213980,132
219108,133
234778,16
278222,169
402693,3
515067,38


In [21]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,route_short_name,direction_id
0,32828689,1,1,5,15,13466640.0,2,5,16,13466531.0,506,0
1,32828689,2,2,5,16,13466531.0,3,5,16,13466677.0,506,0
2,32828689,3,3,5,16,13466677.0,4,5,17,13466754.0,506,0
3,32828689,4,4,5,17,13466754.0,5,5,18,13466699.0,506,0
4,32828689,5,5,5,18,13466699.0,6,5,19,14043045.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213714,32970488,6,7,3,4,13461421.0,8,3,4,13461602.0,306,1
1213715,32970488,7,8,3,4,13461602.0,9,3,5,13461912.0,306,1
1213716,32970488,8,9,3,5,13461912.0,10,3,5,13462284.0,306,1
1213717,32970488,9,10,3,5,13462284.0,11,3,6,13462427.0,306,1


In [22]:
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [23]:
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']])

In [24]:
stop_times_od 

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,route_short_name,direction_id
0,32828689,1,1,5,15,13466640.0,2,5,16,13466531.0,506,0
1,32828689,2,2,5,16,13466531.0,3,5,16,13466677.0,506,0
2,32828689,3,3,5,16,13466677.0,4,5,17,13466754.0,506,0
3,32828689,4,4,5,17,13466754.0,5,5,18,13466699.0,506,0
4,32828689,5,5,5,18,13466699.0,6,5,19,14043045.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213714,32970488,6,7,3,4,13461421.0,8,3,4,13461602.0,306,1
1213715,32970488,7,8,3,4,13461602.0,9,3,5,13461912.0,306,1
1213716,32970488,8,9,3,5,13461912.0,10,3,5,13462284.0,306,1
1213717,32970488,9,10,3,5,13462284.0,11,3,6,13462427.0,306,1


In [25]:
def graph_builder(hr_list, period):
    
    stop_times_od_func = stop_times_od[stop_times_od['hr_o'].isin(hr_list)].copy()
    
    route_map_links = stop_times_od_func.groupby(['INT_ID_o', 'INT_ID_d']
                     ).count().reset_index()[['INT_ID_o','INT_ID_d', 'trip_id']]
    route_map_links = route_map_links.rename(columns = {'trip_id':'freq'})
    
    # adding a dummy link so networkx scales properly
    route_map_links = route_map_links.append(pd.DataFrame.from_records([[1,2, max_freq]], columns = ['INT_ID_o', 'INT_ID_d','freq']))
    route_map_links = route_map_links.reset_index().drop(columns = ['index'])
    
    route_map_links['freq'] = route_map_links['freq']/2
    
    route_map_links['scaled_freq'] = route_map_links['freq']/max_freq
    
    route_map_links['INT_ID_o'] = route_map_links['INT_ID_o'].astype(int)
    route_map_links['INT_ID_d'] = route_map_links['INT_ID_d'].astype(int)
    
    route_map_links['link'] =  route_map_links['INT_ID_o'].astype(str) + '-' + route_map_links['INT_ID_d'].astype(str)
    
    G = nx.DiGraph()
    
    nodes = list((route_map_links['INT_ID_o'].append(route_map_links['INT_ID_d'])).drop_duplicates())
    
    G.add_nodes_from(nodes)
    
    link_list = []
    for index, row in route_map_links.iterrows():
        attributes = {'link_id':row['link'], 
                      'freq':row['freq'], 'scaled_freq': row['scaled_freq']}
        link = (row['INT_ID_o'], row['INT_ID_d'], attributes)
        link_list.append(link)
        
    G.add_edges_from(link_list)
    
    nx.write_gexf(G, 'networks/' + period + '-RM-16-singleDi.gexf')
    
    route_map_links.to_csv('networks/' + period + '-RM-16-singleDi.csv', index = False)

In [26]:
period_list = [['EM', [3,4]], ['AM', [7,8]], ['MD', [11,12]], ['PM', [16,17]], ['EV', [20,21]]]

In [27]:
for i in period_list:
    hr_list = i[1]
    graph_builder(hr_list, i[0])